<a href="https://colab.research.google.com/github/tackulus/229352_BOOTCAMP/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import normaltest

In [2]:
!git clone https://github.com/tackulus/229352_BOOTCAMP.git

Cloning into '229352_BOOTCAMP'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [3]:
data01 = pd.read_csv('/content/229352_BOOTCAMP/DATA/data01.csv', index_col=0)
data02 = pd.read_csv('/content/229352_BOOTCAMP/DATA/data02.csv', index_col=0)
data = pd.concat([data01, data02])
data.head(3)

,year,month,day,maximum temperature(C),minimum temperature(C),dry temperature(C),relative humidity(%),maximum wind direction(Deg.),maximum wind speed(Kts.),wind above the tray(Km.),water evaporation tray(mm.),precipitation(mm.),station,super low temperature grass(C),The maximum surface temperature of the water(C),Minimum surface temperature of the water(C),average temperature(C)
0,1993,1,1,30.3,18.0,22.8,81.0,240.0,5.0,30.0,1.3,0.0,BhumibolDam,NaN,NaN,NaN,NaN
1,1993,1,2,30.5,18.9,23.6,79.0,90.0,8.0,31.0,4.9,0.0,BhumibolDam,NaN,NaN,NaN,NaN
2,1993,1,3,31.7,18.9,23.9,74.0,270.0,6.0,32.0,2.6,0.0,BhumibolDam,NaN,NaN,NaN,NaN


In [8]:
drop_ratio = data.isna().sum() / len(data)
drop_ratio[drop_ratio < 0.2]

year                            0.000000
month                           0.000000
day                             0.000000
maximum temperature(C)          0.032227
minimum temperature(C)          0.034280
dry temperature(C)              0.150821
relative humidity(%)            0.150751
maximum wind direction(Deg.)    0.147925
maximum wind speed(Kts.)        0.147747
water evaporation tray(mm.)     0.051383
precipitation(mm.)              0.144023
station                         0.000000
dtype: float64

In [18]:
def rearrange(feature):
    dst = data[['year', 'month', 'day']].drop_duplicates()
    df = data.set_index(['station', 'year', 'month', 'day'])[feature]

    station_names = data['station'].unique()
    for station in station_names:
        dst[station] = df.loc[station].to_numpy()

    return dst

def fill_NaN(df):
    dst = df.copy()

    for col in dst.columns:
        stat, p = normaltest(dst[col].dropna())
        alpha = 0.05
        if p > alpha: 
            dst[col].fillna(dst[col].mean(), inplace=True)
        else: 
            dst[col].fillna(dst[col].median(), inplace=True)
    
    return dst

dct = dict()
features = ['maximum temperature(C)',
            'minimum temperature(C)', 
            'dry temperature(C)', 
            'relative humidity(%)', 
            'maximum wind direction(Deg.)', 
            'maximum wind speed(Kts.)',
            'water evaporation tray(mm.)', 
            'precipitation(mm.)']

for i,f in enumerate(features):
    df = rearrange(f)
    dct[i] = fill_NaN(df)

In [19]:
dct[0].to_csv("01_max_temperature.csv")
dct[1].to_csv("02_min_temperature.csv")
dct[2].to_csv("03_dry_temperature.csv")
dct[3].to_csv("04_reletive_humidity.csv")
dct[4].to_csv("05_max_wind_direction.csv")
dct[5].to_csv("06_max_wind_speed.csv")
dct[6].to_csv("07_water_eveporation_tray.csv")
dct[7].to_csv("08_precipitation.csv")